In [1]:
#Import libraries and set some options for displaying things
import numpy as np
import scipy.sparse as sp #may not be necessary
import scipy.sparse.linalg as sl #may not be necessary
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from numba import jit #Using jit to improve runtime. To install do conda install numba in cmd
import matplotlib.animation as animation #For making gifs, not required
#import ffmpeg #For gifs
import time
from numpy.linalg import norm

## Boundary conditions and analytic solutions

In [2]:
#Different boundary conditions

@jit
def boundary1(x): #Boundary condition for task b
    return np.cos(4*np.pi*x)

@jit
def boundary2(x): #Second boundary condition for task b. Made periodic for testing
    a=x.copy()
    for i in range(len(a)):
        if a[i]>=1:
            a[i] -=1
    return np.exp(-100*(a-0.5)**2)

@jit
def boundary3(x, y): #Boundary for task c
    return np.outer(np.cos(4*np.pi*x),np.sin(4*np.pi*y))

@jit
def boundary4(x,y):
    return np.outer(np.exp(-100*(x-0.5)**2),np.exp(-100*(y-0.5)**2))

@jit
def solution1(x, t): #Analytic solution for task b
    return np.cos(4*np.pi*(x))*np.cos(4*np.pi*t)

@jit
def solution3(x, y, t, c=1, mesh=True): #Analytic solution for task c. Returns mesh for plotting and error estimates
    if mesh:
        return np.outer(np.cos(4*np.pi*x),np.sin(4*np.pi*y))*np.cos(c*4*np.pi*t*np.sqrt(2))
    return np.cos(4*np.pi*x)*np.sin(4*np.pi*y)*np.cos(c*4*np.pi*t*np.sqrt(2))


## 1D wave equation solver

In [3]:
@jit(nopython=True) #Jit for faster compilation
def wave1D(Nx, Nt, c, boundfnc, t0 = 0, T = 1, x0 = 0, X = 1):
    '''
    Description: Solves the 1D-wave equation with Neumann, Dirichlet and periodic boundary
                 conditions. Note that the Neumann condition u_t(x,0)=0 is hard coded.
    
    Input
    Nx: Number of points in x-direction
    Nt: Number of points in t-direction
    c: Wave velocity. Dimensionless
    boundfnc: Boundary function for u(x,0)
    t0: Starting time. Default 0
    T: End time. Default 1
    x0: Starting position. Default 0
    X: End position. Default 1
    
    Returns:
    solution: (Nt+1) X (Nx+1) matrix containing the solution
    xaxis: A Nx+1 vector of x values. Used for plotting and error estimates
    haxis: A Nt+1 vector of time values. Used for plotting and error estimates
    '''
    
    #Initialize variables
    h = (X-x0)/(Nx+1) #Step size x
    k = (T-t0)/(Nt+1) #Step size t
    
    #Checking that r reaches our requirment.
    if(c*k/h>1):
        raise Exception("r does not reach the requirment |r|<=1")
    
    #Create axies for plotting and such
    xaxis = np.arange(x0,X,h)[: Nx+1] #Axis for points to be evaluated for g(x)
    haxis = np.arange(t0, T, k) #Axis for points in h. For plotting
    
    #Courant number squared
    C = k**2*c**2/h**2 
    
    #Define vectors
    up = np.zeros(Nx+1) #Solution at X_{t+1}
    um = np.zeros(Nx+1) #Solution at X_{t-1}
    u = np.zeros(Nx+1) #Solution at X_{t}
    sol = np.zeros((Nt+1, Nx+1)) #Solution matrix, stores the final result
    um = boundfnc(xaxis) #Get initial conditions from boundary function
    sol[0] = um #Store solution
    
    #Begin with the first special iteration
    for i in range(1,Nx):
        u[i] = um[i] + 0.5*C*(um[i+1] - 2*um[i] + um[i-1])
    
    #Impose the periodic boundary conditions
    u[0] = um[0] + 0.5*C*(um[1] - 2*um[0] + um[-2]) 
    u[-1] = um[-1] + 0.5*C*(um[1] - 2*um[-1] + um[-2]) 
    
    #Iterate through time
    for i in range(1,Nt+1):
        #Iterate through space
        for j in range(1,Nx):
            up[j] = 2*u[j] - um[j] + C*(u[j+1] - 2*u[j] + u[j-1])
        
        #Encforce periodic boundary conditions
        up[0] = 2*u[0] - um[0] + C*(u[1] - 2*u[0] + u[-2])
        up[-1] = 2*u[-1] - um[-1] + C*(u[1] - 2*u[-1] + u[-2])
        
        #Store solution, prepare for next iterate
        sol[i] = u.copy()
        
        #Swap variables
        um = u.copy()
        u = up.copy()
    
    #Return the solution with axies.
    return sol.transpose(),xaxis,haxis

## 2D wave equation solver

In [4]:
@jit(nopython=True) #Jit for faster compilation
def wave2D(Nx, Ny, Nt, boundfnc, c, t0 = 0, T = 1, x0 = 0, X = 1, y0 = 0, Y = 1):
    '''
    Description: Solves the 2D-wave equation with Neumann, Dirichlet and periodic boundary
                 conditions. Note that the Neumann condition u_t(x,y,0)=0 is hard coded.
    
    Input
    Nx: Number of points in x-direction
    Ny: Number of points in y-direction
    Nt: Number of points in t-direction
    c: Wave velocity. Dimensionless
    boundfnc: Boundary function for u(x,0)
    t0: Starting time. Default 0
    T: End time. Default 1
    x0: Starting position in x. Default 0
    X: End position in x. Default 1
    y0: Starting position in y. Default 0
    Y: End position in y. Default 1
    
    Returns:
    solution: (Nt+1) X (Nx+1) X (NY+1) matrix containing the solution
    xaxis: A Nx+1 vector of x values. Used for plotting and error estimates
    yaxis: A Ny+1 vector of y values. Used for plotting and error estimates
    haxis: A Nt+1 vector of time values. Used for plotting and error estimates
    r
    '''

    #Initialize some variables
    h1 = (X-x0)/(Nx+1) #Stepsize in x
    h2 = (Y-y0)/(Ny+1) #Stepsize in y
    k = (T-t0)/(Nt+1) #Stepsize in t
    xaxis = np.arange(x0, X+h1, h1)[:Nx+2] #Axis for points to be evaluated for u(x,y,0)
    yaxis = np.arange(y0, Y+h2, h2)[:Ny+2] #Axis for points to be evaluated for u(x,y,0)
    haxis = np.arange(t0, T, k) #Axis for points in t. For plotting
    
    #Checking that r satisfies the stability requirment.
    r = (c*k/h1)**2 + (c*k/h2)**2
    if r > 1:
     #  raise Exception("r does not reach the requirment |r|<=1", r)
        print("Warning, r>1!",r)
        
    #Courant numbers squared
    C1 = k**2*c**2/h1**2 
    C2 = k**2*c**2/h2**2 
    
    #Define vectors
    up = np.zeros((Nx+2, Ny+2)) #Stores the solution at U_{t+1}
    um = np.zeros((Nx+2, Ny+2)) #Stores the solution at U_{t}
    u = np.zeros((Nx+2, Ny+2)) #Stores the solution at U_{t-1}
    sol = np.zeros((Nt+1, Nx+2, Ny+2)) #Solution Matrix stores the final result
    um = boundfnc(xaxis, yaxis) #Get initial conditions.
    sol[0] = um #Store solution
    
    #First special iteration
    for i in range(1,Nx+1):
        for l in range(1, Ny+1):
            ux = um[i-1,l] - 2*um[i,l] + um[i+1,l]
            uy = um[i,l-1] - 2*um[i,l] + um[i,l+1]
            u[i,l] = um[i,l] + 0.5*C1*ux + 0.5*C2*uy
    
    #Enforce periodic boundary conditions in corners
    u[0,0] = um[0,0] + 0.5*C1*(um[1,0] - 2*um[0,0] + um[-2,0]) + 0.5*C2*(um[0,1] - 2*um[0,0] + um[0,-2]) 
    u[-1,0] = u[0,0]
    u[0,-1] = u[0,0]
    u[-1,-1] = u[0,0]
    
    #Enforce periodic boundary conditions on x
    for i in range(1,Nx+1):
        u[i,0] = um[i,0] + 0.5*C1*(um[i+1,0] - 2*um[i,0] + um[i-1,0]) + 0.5*C2*(um[i,1] - 2*um[i,0] + um[i,-2]) 

    #Enforce periodic boundary conditions on y
    for i in range(1,Ny+1):
        u[0,i] = um[0,i] + 0.5*C1*(um[1,i] - 2*um[0,i] + um[-2,i]) + 0.5*C2*(um[0,i+1] - 2*um[0,i] + um[0,i-1])
    
    #Copy to other side
    u[-1,:] = u[0,:].copy()
    u[:,-1] = u[:,0].copy()
    
    #Iterate through time
    for j in range(1,Nt+1):
        #Iterate through space
        for i in range(1,Nx+1):
            for l in range(1,Ny+1):
                ux = u[i-1,l] - 2*u[i,l] + u[i+1,l]
                uy = u[i,l-1] -2*u[i,l] + u[i,l+1]
                up[i,l] = 2*u[i,l] - um[i,l] + C1*ux + C2*uy
        
        #Enforce periodic boundary conditions in corners
        up[0,0] = 2*u[0,0] - um[0,0] + C1*(u[1,0] - 2*u[0,0] + u[-2,0]) + C2*(u[0,1] - 2*u[0,0] + u[0,-2]) 
        up[-1,0] = up[0,0]
        up[0,-1] = up[0,0]
        up[-1,-1] = up[0,0]
        
        #Enforce periodic boundary conditions on x
        for i in range(1,Nx+1):
            up[i,0] = 2*u[i,0] - um[i,0] + C1*(u[i+1,0] - 2*u[i,0] + u[i-1,0]) + C2*(u[i,1] - 2*u[i,0] + u[i,-2]) 

        #Enforce periodic boundary conditions on y
        for i in range(1,Ny+1):
            up[0,i] = 2*u[0,i] - um[0,i] + C1*(u[1,i] - 2*u[0,i] + u[-2,i]) + C2*(u[0,i+1] - 2*u[0,i] + u[0,i-1])
        
        #Copy to other side
        up[-1,:] = up[0,:].copy()
        up[:,-1] = up[:,0].copy()
        
        #Store solution, prepare for next iterate
        sol[j] = u.copy()
        
        #Swap variables
        um = u.copy()
        u = up.copy()
    
    #Add the last computed solution
    sol[-1] = up
    
    #Return the values.
    return sol, xaxis, yaxis, haxis, r

## Plotting functions

In [5]:
def makeGIF1D(solution, xaxis, fps = 10, frn = 50, save=False): #Function to make gifs for the 1D wave equation
    N=150 #Meshsize
    
    fig, ax = plt.subplots() #Obtain figure

    line, = ax.plot(xaxis, solution[0]) #Plot the first time step

    #Animate function. Used to proceed to next image.
    def animate(i):
        line.set_ydata(solution[i])  # update the data.
        return line,

    #Make animation
    ani = animation.FuncAnimation(
        fig, animate, frn, interval=1000/fps, blit=False, save_count=50)

    # To save the animation
    if save:
        ani.save("1D wave.gif", fps=fps)
    
    #Display animation
    plt.show()

    
def makeGIF2D(zarray,x,y, save=False): #Plot for making gifs of solutions for the 2D wave equation
    N = 150 # Meshsize
    fps = 30 # frame per sec
    frn = len(zarray) # frame number of the animation
    fig = plt.figure() #Obtain a figure

    ax = fig.add_subplot(111, projection='3d') #Add 3d axis
    
    #Animate function. Used to proceed to next image.
    def update_plot(frame_number, zarray, plot):
        plot[0].remove()
        plot[0] = ax.plot_surface(x, y, zarray[frame_number,:,:],cmap="viridis",vmin=-1.1, vmax=1.1)
        
    #Make mesh axis
    x = np.outer(x, np.ones(len(y)))
    t = np.outer(y, np.ones_like(x)).T

    #Plot
    plot = [ax.plot_surface(x, y, zarray[0,:,:], color='0.75', rstride=1, cstride=1)]
    ax.set_zlim(-1.1,1.1) #Limit
    
    #Obtain animation
    ani = animation.FuncAnimation(fig, update_plot, frn, fargs=(zarray, plot), interval=1000/fps)
    
    #potentially save gif
    if save:
        ani.save('2D wave.gif',writer='imagemagick',fps=fps)
    
    #Display animation
    plt.show()

def displaySlice(xaxis,solution, timeframe, ylim = None):
    assert timeframe<=len(solution)
    
    fig, ax = plt.subplots()
    
    ax.plot(xaxis,solution[timeframe], label = "Solution at t=$"+str(np.round(timeframe/len(solution),2)) +"$", marker = ".")
    ax.legend()
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    if(ylim!=None):
        ax.set_ylim(ylim)
    ax.grid()
    plt.show()
    
    
def displaySlice3d(xaxis, yaxis, solution, timeframe, zlim = None,title = "",vmin = None, vmax = None):
    if callable(solution):
        plot3D(solution(xaxis,yaxis,t), xaxis, yaxis, zlabel = "$u(x,y,t)$",zlim = zlim, title = title, vmin = vmin, vmax = vmax)
    else:
        assert timeframe <=len(solution)
        plot3D(solution[timeframe], xaxis, yaxis, zlabel = "$U(x,y,t)$",zlim = zlim, title = title, vmin = vmin, vmax = vmax)
    

In [6]:
#Function that makes errorplot for b)
def errorPlot1D(startIndex = 1, endIndex = 3.3, evaluationPoints = 5,r = 2**(-1), returnValues = True):
    
    M = np.logspace(startIndex, endIndex, evaluationPoints)
    
    errors = np.zeros(len(M))
    i = 0
    for m in M:
        k = r*1/(m+1)
        
        numSol, xaxis, haxis = wave1D(int(m), int(1/k)-1, 1, boundary1)
        
        a = solution1(xaxis, 1)
        
        errors[i] = np.linalg.norm(numSol[:,-1]-a)/np.linalg.norm(a)
        i+=1
    
    plt.loglog(M, errors, label="$\ell_2$-norm", marker = ".")
    orderx, ordery = expectedOrder(M[0], M[-1], errors[0], 2, resolution=5)
    plt.plot(orderx,ordery, "s",label="Expected order")
    plt.legend()
    plt.xlabel("Number of points along x-axis: $M$")
    plt.ylabel("Relative error $e^r_{(\ell_2)}$")
    plt.grid()
    plt.show()
    
    if(returnValues):
        return errors, M


########################################################################################################
  
#Makes errorplot for task c
def errorPlot2D(startIndex = 1, endIndex = 2.7, evaluationPoints = 10,r_x = 2**(-1), returnValues = True):
    
    #M_x = [i for i in range(startIndex, endIndex, int((endIndex-startIndex)/5))]
    M_x = np.logspace(startIndex, endIndex, evaluationPoints)
    
    errors = np.zeros(len(M_x))
    i = 0
    for mx in M_x:
        
        my = mx
        k = r_x*1/(mx+1)
        
        numSol, xaxis, yaxis, haxis, r = wave2D(int(mx), int(my), int(1/k)-1, boundary3, 1)
        
        a = solution3(xaxis, yaxis, 1)
        
        errors[i]=np.linalg.norm(numSol[-1,:,:]-a)/np.linalg.norm(a)
        i+=1
    
    plt.loglog(M_x, errors, label = "$\ell_2$-norm", marker = ".")
    orderx, ordery = expectedOrder(M_x[0], M_x[-1], errors[0], 2, resolution=5)
    plt.plot(orderx, ordery, "s", label = "Expected order")
    plt.xlabel("Number of points in $x$-direction: $M_x$")
    plt.ylabel("Relative error $e^r_{(\ell_2)}$")
    plt.legend()
    plt.grid()
    plt.show()
    
    if(returnValues):
        return errors, M_x

########################################################################################################

#Makes errorplot for d)
def errorPlotNdof(MNk, c=1, returnValues = True, logscaleTime = False):
    
    #creates arrays for storing degrees of freedom, errors and computational times
    degOfFree = np.zeros(len(MNk))
    compTimes = np.zeros(len(MNk))
    errors = np.zeros(len(MNk))
    
    i= 0
    for mnk in MNk:
        degOfFree[i]=mnk[0]*mnk[1]/mnk[2] #Calculates the degrees of freedom
        
        T0 = time.time() #Starts clock
        solution, xaxis, yaxis, haxis, r = wave2D(int(mnk[0]), int(mnk[1]), int(1/mnk[2])-1, boundary3, 1)
        compTimes[i]=time.time()-T0 #Stores time to solve problem
        
        errors[i] = np.linalg.norm(solution[-1, :, :]-solution3(xaxis, yaxis, 1,1))/np.linalg.norm(solution3(xaxis, yaxis, 1,1))
        i+=1
    
    #Plots the computational time agains degrees of freedom
    fig, ax = plt.subplots()
    ax.plot(degOfFree, compTimes, label = "Computational time", marker = ".")
    if logscaleTime:
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_ylabel("log(seconds)")
    else:
        ax.set_ylabel("seconds")
    ax.set_xlabel("Number of degrees of freedom")
    
    ax.legend()
    ax.grid()
    plt.show()

    #Plot the relative l2 error against degrees of freedom
    fig2, ax2 = plt.subplots()
    ax2.loglog(degOfFree, errors, label = "$\ell_2$-norm", marker = ".")
    orderx, ordery = expectedOrder(degOfFree[0],degOfFree[-1], errors[0],2/3)
    ax2.plot(orderx, ordery, linestyle = (0, (5,10)), label="Expected order")
    ax2.legend()
    ax2.set_xlabel("Number of degrees of freedom")
    ax2.set_ylabel("Relative error $e^r_{(\ell_2)}$")
    ax2.grid()
    plt.show()
    
    if(returnValues):
        return MNk, degOfFree, errors, compTimes
    

In [7]:
def estimateOrder(xvec, yvec, sigfigs = 3):
    if(len(xvec)!=len(yvec)):
        raise Exception("the inputs do not have the same dimension")
    ylog = np.log(yvec)
    xlog = np.log(xvec)
    return np.round(np.abs(np.average(ylog/xlog)), sigfigs)